# 🧠 SHL Grammar Scoring Challenge — LLM-Based Approach 🚀

## 📌 Problem Statement
The goal is to **automatically evaluate the grammatical quality of spoken English sentences**. Given a `.wav` audio file as input, we need to:
- Transcribe the audio into text.
- Score the grammatical accuracy of the sentence (1–5).
- Optionally suggest improvements and highlight mistakes.

## 🔧 Pipeline Overview
Our solution follows a **modular pipeline-based approach** with the following steps:

1. **Transcription** using Whisper.
2. **LLM-Based Scoring** via prompt engineering.
3. **Robust Retry Logic** for unstable transcriptions.
4. **Post-Processing** and formatting into submission.



In [2]:
!pip install Groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 5.6 MB/s eta 0:00:00


In [3]:
import os
import pandas as pd
import time
import re
from groq import Groq
import torchaudio

In [4]:
from google.colab import files
files.upload()  # select kaggle.json

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"shikherjha","key":"3e3ee9365a807d5377b05249aabf96b5"}'}

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle competitions download -c shl-intern-hiring-assessment
!unzip shl-intern-hiring-assessment.zip -d data

Archive:  shl-intern-hiring-assessment.zip
  inflating: data/dataset/audios_test/audio_10.wav  
  inflating: data/dataset/audios_test/audio_1012.wav  
  inflating: data/dataset/audios_test/audio_1013.wav  
  inflating: data/dataset/audios_test/audio_1019.wav  
  inflating: data/dataset/audios_test/audio_1022.wav  
  inflating: data/dataset/audios_test/audio_1026.wav  
  inflating: data/dataset/audios_test/audio_103.wav  
  inflating: data/dataset/audios_test/audio_1033.wav  
  inflating: data/dataset/audios_test/audio_1035.wav  
  inflating: data/dataset/audios_test/audio_1048.wav  
  inflating: data/dataset/audios_test/audio_1054.wav  
  inflating: data/dataset/audios_test/audio_1058.wav  
  inflating: data/dataset/audios_test/audio_1061.wav  
  inflating: data/dataset/audios_test/audio_1068.wav  
  inflating: data/dataset/audios_test/audio_107.wav  
  inflating: data/dataset/audios_test/audio_1081.wav  
  inflating: data/dataset/audios_test/audio_1089.wav  
  inflating: data/dataset/

In [7]:
!ls data/dataset



audios_test  audios_train  sample_submission.csv  test.csv  train.csv


### 📤 Upload Transcribed & Labeled Training Data (`train_Transcript.csv`)

This block handles the upload of the preprocessed training file that includes:
- ✅ Transcribed audio (from Whisper or similar ASR system)
- 🎯 Ground truth grammar scores (labels)
- 🧽 Cleaned and standardized text for reliable few-shot prompt generation

---

#### 📄 File Details:
- **Filename**: `train_Transcript.csv`
- **Columns Expected**:
  - `text`: Cleaned transcriptions from audio
  - `label`: Grammar quality scores (float between 1.0–5.0)

---

#### 📌 Purpose:
This file is crucial for **few-shot prompting**. It helps provide diverse, labeled examples across score ranges to guide the LLM’s grammar evaluation.

Uploading this file enables:
- Dynamic example selection from different score buckets (e.g., 1.0–2.0, 2.0–3.0, etc.)
- More accurate and consistent scoring when evaluating unseen test audio

> ⚠️ Make sure the file is already preprocessed with accurate transcriptions and score alignment. If not, ensure to run the preprocessing pipeline first before uploading.


In [8]:
from google.colab import files
import pandas as pd

# Step 1: Upload CSV file
uploaded = files.upload()  ##UPLOAD train_Trasncript.csv for better prompting

Saving train_transcripts (1).csv to train_transcripts (1).csv


In [9]:
# === CONFIG ===
GROQ_API_KEY = "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"
client = Groq(api_key=GROQ_API_KEY)

In [10]:
# === LOAD TRAIN DATA FOR FEW-SHOT ===
df = pd.read_csv("train_transcripts (1).csv")
few_shot_examples = []


In [11]:
print("CSV Columns:", df.columns.tolist())


CSV Columns: ['text', 'label']


### 🎯 Few-Shot Example Selection for Prompt Engineering

To improve the LLM's grammar evaluation consistency, we construct a balanced few-shot context by selecting representative examples across different score ranges. This helps the model understand the nuances between different quality levels of grammar.

#### ✅ What This Code Does:
- **Cleans the column names** by stripping whitespace and converting to lowercase.
- **Defines 4 score bands**:
  - 1.0–2.0 (Poor),
  - 2.0–3.0 (Below Average),
  - 3.0–4.0 (Good),
  - 4.0–5.0 (Excellent).
- **Samples one sentence from each band** using stratified sampling to ensure diversity in prompt examples.
- Appends each selected sample to a `few_shot_examples` list, which can then be embedded in prompts to guide the LLM scoring behavior.

This step is critical to help the model learn how grammar quality varies across scores and calibrate predictions accordingly.


In [12]:
# Sanitize and lowercase column names to be safe
df.columns = df.columns.str.strip().str.lower()

# Clear previous examples if any
few_shot_examples = []

# Define score ranges for better few-shot diversity
score_ranges = [(1.0, 2.0), (2.0, 3.0), (3.0, 4.0), (4.0, 5.1)]  # Note: 5.1 to include score = 5.0

for low, high in score_ranges:
    filtered = df[(df['label'] >= low) & (df['label'] < high)]
    if not filtered.empty:
        sample = filtered.sample(1).iloc[0]
        few_shot_examples.append({
            "sentence": sample["text"],
            "score": round(float(sample["label"]), 2)
        })


In [13]:
def transcribe_audio_whisper(audio_path):
    with open(audio_path, "rb") as audio_file:
        response = client.audio.transcriptions.create(
            file=audio_file,  # <== FILE OBJECT, not bytes
            model="whisper-large-v3",
            response_format="text"
        )
    return response


### 🧠 Prompt Construction for LLM Grammar Evaluation

The `build_prompt()` function is designed to guide the language model in scoring a given sentence based on grammar. This function leverages few-shot learning by embedding diverse labeled examples, improving the model's calibration and reliability.

#### 🔍 Structure of the Prompt:
1. **Introduction & Guidelines**  
   - Explains the task: grammar evaluation on a 1.0 to 5.0 scale.
   - Defines clear criteria for each score range.
   - Specifies the required output format:  
     ```
     Score: <float>
     Suggestions: <tips>
     Mistakes: <grammar issue tags>
     ```

2. **Few-Shot Examples**  
   - Dynamically inserts 1 sentence per score range (1–5), with its corresponding score.
   - These help the model infer how different grammar qualities should be rated.

3. **Final Sentence to Evaluate**  
   - Appends the test sentence that the model needs to score.

#### ✨ Example Output Prompt Structure:


In [14]:
# === PROMPT BUILDING FUNCTION ===
def build_prompt(test_sentence):
    intro = (
        "You're a grammar evaluation model. You will analyze the grammar of English sentences and assign a score from 1.0 to 5.0, where:\n"
        "- 1.0–2.0 → Poor grammar with multiple errors affecting clarity.\n"
        "- 2.0–3.0 → Fair grammar, with several noticeable errors but understandable.\n"
        "- 3.0–4.0 → Good grammar, minor errors, mostly clear and correct.\n"
        "- 4.0–5.0 → Excellent grammar, very few or no mistakes.\n\n"
        "Respond in this format:\n"
        "Score: <float score>\n"
        "Suggestions: <grammar improvement tips>\n"
        "Mistakes: <grammar tags like tense, article, punctuation, etc.>\n\n"
        "Here are examples:\n\n"
    )
    examples = ""
    for ex in few_shot_examples:
        examples += (
            f"Sentence: \"{ex['sentence']}\"\n"
            f"Score: {ex['score']}\n"
            f"Suggestions: <example suggestion>\n"
            f"Mistakes: <example tags>\n\n"
        )

    final_input = f"Now evaluate:\nSentence: \"{test_sentence}\"\n"
    return intro + examples + final_input


In [15]:
# === SCORE EXTRACTOR FROM LLM RESPONSE ===
def extract_response_parts(response_text):
    score_match = re.search(r"Score:\s*([0-5](?:\.\d+)?)", response_text)
    suggestions_match = re.search(r"Suggestions:\s*(.*)", response_text)
    mistakes_match = re.search(r"Mistakes:\s*(.*)", response_text)

    score = float(score_match.group(1)) if score_match else None
    suggestions = suggestions_match.group(1).strip() if suggestions_match else "N/A"
    mistakes = mistakes_match.group(1).strip() if mistakes_match else "N/A"

    return score, suggestions, mistakes

In [16]:
import os
import pandas as pd
from tqdm import tqdm
import time
import random

### 🔁 Audio Evaluation Pipeline (`evaluate_audio_file`)

This function takes a **WAV audio file** as input and performs the full pipeline of:
1. **Transcription** (via Whisper ASR)
2. **Grammar Scoring** (via LLaMA3 LLM)
3. **Parsing Model Output** into structured components

---

#### 🔨 Step-by-Step Breakdown:

1. **Transcription using Whisper**  
   - The raw audio is passed to the Whisper ASR engine (`transcribe_audio_whisper()`).
   - This step extracts the spoken sentence into text.

2. **Prompt Construction**  
   - The transcribed text is embedded in a prompt using the `build_prompt()` function.
   - The prompt includes grading criteria and few-shot examples to help the LLM understand how to score grammar.

3. **Evaluation with LLaMA3**  
   - The constructed prompt is sent to the `llama3-8b-8192` model via Groq's API.
   - Temperature is set to 0.3 to maintain deterministic and stable outputs.

4. **Response Parsing**  
   - The raw output from the model is parsed using `extract_response_parts()`.
   - Extracted fields:  
     - `score`: A float between 1.0 and 5.0  
     - `suggestions`: Grammar improvements  
     - `mistakes`: Tags of grammar issues (e.g., tense, punctuation)

5. **Rate Limiting**  
   - A `time.sleep(1.2)` delay ensures the API isn't overwhelmed (especially useful when running in batch mode).

---

#### 🧾 Return Value:
A Python dictionary with:
```python
{
  "transcription": "The transcribed sentence",
  "score": float,
  "suggestions": "Improvement tips",
  "mistakes": "Grammar issue tags"
}


In [17]:
def evaluate_audio_file(audio_path):
    print(f"[+] Transcribing {audio_path} ...")
    transcribed_text = transcribe_audio_whisper(audio_path)
    print(f"[✓] Transcribed: {transcribed_text}")

    prompt = build_prompt(transcribed_text)
    print("[+] Sending to LLM for evaluation...")

    chat_response = client.chat.completions.create(
        model="llama3-8b-8192",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
    )
    llm_output = chat_response.choices[0].message.content
    print(f"[✓] LLM Output:\n{llm_output}")

    score, suggestions, mistakes = extract_response_parts(llm_output)
    time.sleep(1.2)  # Add this just before the end of evaluate_audio_file

    return {
        "transcription": transcribed_text,
        "score": score,
        "suggestions": suggestions,
        "mistakes": mistakes
    }


### 🔄 Robust Audio Evaluation with Retry Mechanism (`robust_evaluate_audio`)

This wrapper function ensures **resilience** when interacting with external services (like Whisper or LLM APIs) that are prone to **rate limits**, **timeouts**, or other transient errors.

---

#### 🔐 Purpose:
To handle occasional failures gracefully (especially rate-limiting errors) by **retrying** the audio evaluation a few times before completely giving up.

---

#### 🔧 How It Works:

1. **Retry Logic**  
   - The function will try to call `evaluate_audio_file(audio_path)` up to `MAX_RETRIES` times.
   - By default, `MAX_RETRIES = 5`.

2. **Rate Limit Detection & Exponential Backoff**  
   - If the caught error contains `"429"` or mentions `"rate"` in any form (case-insensitive), it's assumed to be a rate limit.
   - The retry then waits using **exponential backoff** + some jitter:
     ```
     wait_time = 2^attempt + random(0, 2)
     ```

3. **Immediate Exception Propagation**  
   - If the error is **not a rate limit**, it's immediately raised, preserving original stack trace and debugging info.

4. **Final Failure Handling**  
   - After all retries fail, a `RuntimeError` is raised with the failed audio path.

---

#### ✅ Benefits:
- **Avoids losing data** due to transient issues.
- **Prevents API ban** by spacing requests intelligently.
- Makes batch processing **more robust and stable**.

---

#### ⚠️ Important:
Ensure this function wraps **all calls to `evaluate_audio_file()`** in bulk processing loops.



In [18]:
MAX_RETRIES = 5

def robust_evaluate_audio(audio_path):
    for attempt in range(MAX_RETRIES):
        try:
            return evaluate_audio_file(audio_path)
        except Exception as e:
            if "429" in str(e) or "rate" in str(e).lower():
                wait_time = 2 ** attempt + random.uniform(0, 2)
                print(f"[!] Rate limit hit. Retrying in {wait_time:.1f}s... (Attempt {attempt+1}/{MAX_RETRIES})")
                time.sleep(wait_time)
            else:
                raise e
    raise RuntimeError(f"Exceeded retries for {audio_path}")

### 🧪 Batch Evaluation of Test Audio Files (`evaluate_all_test_audios`)

This function processes all `.wav` files in a specified test directory using the previously defined `robust_evaluate_audio()` function. It builds the final test submission DataFrame by transcribing, evaluating grammar, and capturing any issues during inference.

---

#### 📦 Input:
- `test_audio_dir` (str): Path to the folder containing test `.wav` audio files.

---

#### ⚙️ Workflow:
1. **Discovery**  
   - Uses `os.listdir()` to find all `.wav` files and sort them for consistent order.

2. **Batch Inference with Progress**  
   - Iterates through each test audio file using `tqdm` to show live progress.
   - Calls `robust_evaluate_audio()` to:
     - Transcribe the audio
     - Send the transcript to LLM
     - Extract score, suggestions, and grammar mistakes

3. **Error Handling**  
   - If evaluation fails even after retrying, it logs the error and fills the row with:
     - Empty transcription
     - `None` for score
     - Error message in `suggestions`
     - Empty `mistakes`

4. **Result Collection**  
   - Stores results in a list of dictionaries.
   - Converts the list into a `pandas.DataFrame` for downstream saving or submission.

---

#### 📤 Output:
- A DataFrame with the following columns:
  - `audio_file`: filename of the input audio
  - `transcription`: Whisper transcription output
  - `score`: predicted grammar score (float or `None`)
  - `suggestions`: improvement suggestions or error message
  - `mistakes`: grammar categories identified by the model

---

#### ✅ Why It’s Useful:
- Automates evaluation for all test samples.
- Ensures fault tolerance so a few failing files don’t break the entire pipeline.
- Organizes model outputs in a structured format ready for Kaggle submission.

---

#### 📌 Tip:
Always verify that your final DataFrame:
- Matches the expected schema.
- Has `score` values in `float` type.
- Handles all errors gracefully.


In [19]:


def evaluate_all_test_audios(test_audio_dir):
    results = []
    audio_files = sorted([f for f in os.listdir(test_audio_dir) if f.endswith(".wav")])
    print(f"[+] Found {len(audio_files)} test audio files.")

    for audio_file in tqdm(audio_files, desc="Evaluating Test Set"):
        audio_path = os.path.join(test_audio_dir, audio_file)

        try:
            result = robust_evaluate_audio(audio_path)
            result['audio_file'] = audio_file
            results.append(result)

        except Exception as e:
            print(f"[X] Error on {audio_file}: {e}")
            results.append({
                "audio_file": audio_file,
                "transcription": "",
                "score": None,
                "suggestions": f"Error: {str(e)}",
                "mistakes": ""
            })

    return pd.DataFrame(results)


In [20]:
df_results = evaluate_all_test_audios("data/dataset/audios_test/")

# Save for leaderboard or review
df_results[["audio_file", "score"]].to_csv("submission.csv", index=False)
print("[✓] Saved as submission.csv")

# Save full results with suggestions
df_results.to_csv("full_output.csv", index=False)
print("[✓] Full output saved as full_output.csv")


[+] Found 195 test audio files.


Evaluating Test Set:   0%|          | 0/195 [00:00<?, ?it/s]

[+] Transcribing data/dataset/audios_test/audio_10.wav ...
[✓] Transcribed:  The scene of the playground was filled with children. They were laughing and screaming and running around with lots of energy. There was a big open field, although some of it was tarmac and some of it had greenery. There was lots of games to play. I seen some kids playing hopscotch. I seen some kids playing with skipping ropes. There was even slides and swings. It looked as if it was in full force. There were some parents on the side watching the kids play in the playground. And there were some people walking past as they looked in at the children having lots of fun in the playground. Amen.
[+] Sending to LLM for evaluation...
[✓] LLM Output:
Score: 3.2
Suggestions: Consider using more precise vocabulary, such as "numerous" instead of "lots of" to describe the games and activities. Also, try to vary sentence structure and length to improve overall flow and clarity.

Mistakes:

* Tense inconsistency: The senten

Evaluating Test Set:   1%|          | 1/195 [00:05<17:16,  5.34s/it]

[+] Transcribing data/dataset/audios_test/audio_1012.wav ...
[✓] Transcribed:  The playground looks like beautiful in that we will enjoy and we are ready to play and our stresses decreases in playground. Full joy and enjoyment will be there in that playground and we will play several games like outdoor games like cricket and badminton and etc in that playground and there is no rules and regulations in the playground whatever you like to do in a good manner in a particular doing is good for health and playing games is also good for health.
[+] Sending to LLM for evaluation...
[✓] LLM Output:
Score: 2.0
Suggestions: Break up long sentences into shorter ones, and consider rephrasing some of the sentences to improve clarity and coherence.

Mistakes:

* Lack of subject-verb agreement (e.g., "we will enjoy and we are ready to play")
* Inconsistent verb tense (e.g., "will enjoy" vs. "are ready")
* Use of vague pronouns (e.g., "that" instead of specifying which playground)
* Lack of clear tran

...

[+] Transcribing data/dataset/audios_test/audio_971.wav ...
[✓] Transcribed:  The school playground is composed of students who are playing together or roaming around. The students are usually playing in the playground after their classes or if the class are cancelled and they are waiting for their parents or guardian to fetch them. The playground includes some swing and other activities that the student would enjoy. and or or maybe it is or maybe it is it is a ground where where the children can can roam around
[+] Sending to LLM for evaluation...
[✓] LLM Output:
Score: 2.0
Suggestions: Break up long sentences into shorter ones, and use more precise language to convey your ideas. For example, instead of "The playground includes some swing and other activities that the student would enjoy. and or or maybe it is or maybe it is it is a ground where where the children can can roam around", consider "The playground offers swings and other activities that students enjoy. It's also a space w

Evaluating Test Set:  99%|█████████▉| 194/195 [52:53<00:28, 28.98s/it]

[+] Transcribing data/dataset/audios_test/audio_998.wav ...
[✓] Transcribed:  In a backdrop of a school, you might see bustling hallways filled with students hurrying to their next class. Colorful bulletin boards adorned with students' artwork and academic achievements. The teachers engage in animated discussions with their students. the faint sound bells signaling the fight of the end classes and perhaps courtyard and playground of students gather the information in all the nations all to then gather during ground where students gather during breaks in typical school playground scenes you might envision children running around laughing and playing various games you see swings swaying back and forth the rhythmic clanking of chains against metal poles, while others might be climbing on a jungle gym or playing tag. There's often a...
[+] Sending to LLM for evaluation...
[✓] LLM Output:
Score: 4.0
Suggestions: Consider breaking up long sentences into shorter ones for better clarity and fl

Evaluating Test Set: 100%|██████████| 195/195 [53:32<00:00, 16.48s/it]

[✓] Saved as submission.csv
[✓] Full output saved as full_output.csv


In [38]:

df_results.rename(columns={"audio_file": "filename"}, inplace=True)
df_results.rename(columns={"score": "label"}, inplace=True)


In [39]:
invalid_rows = df_results[df_results["label"].isna() | (df_results["label"] == "")]
invalid_rows


,filename,label


### 🔁 Retrying Failed Audio Evaluations

This block re-attempts evaluation for test audio files that previously failed during batch inference. It ensures we extract a valid `score` even for files that encountered transient issues like rate limits, network timeouts, or model errors.

---

#### 📦 Input:
- `invalid_rows`: A subset of `df_results` where grammar score (`label`) is `None` or `NaN`, indicating failure.
- `df_results`: The full results DataFrame to be updated upon successful retries.

---

#### ⚙️ Workflow:
1. **Iterate Over Failed Rows**  
   - Loop through each failed entry using `tqdm` for visual progress.
   - Extract the audio filename and reconstruct the full path.

2. **Retry Mechanism (Up to 3 Attempts)**  
   - Tries to re-evaluate the audio using `evaluate_audio_file`.
   - Implements **exponential backoff** with jitter to avoid triggering rate limits.

3. **Success Handling**  
   - If a retry succeeds, the score (`label`) is updated in `df_results` at the correct row index.

4. **Failure Handling**  
   - After all attempts, if evaluation still fails, it logs the filename for manual inspection.

---

#### ✨ Features:
- **Resilient Execution**: Recovers from transient failures without restarting the entire pipeline.
- **Selective Update**: Only updates the `label` (score) to avoid overwriting other valid fields like transcription or suggestions.
- **Exponential Backoff**: Helps mitigate API rate limit issues and stabilizes performance during retry loops.

---

#### 📌 Tip:
- You can enhance this by also updating transcription/suggestions if missing.
- After retrying, consider saving `df_results` again to ensure all scores are stored for submission.


In [34]:
import time
import random
from tqdm import tqdm

for idx, row in tqdm(invalid_rows.iterrows(), total=len(invalid_rows), desc="Retrying failed audios"):
    filename = row["filename"]
    audio_id = filename.replace(".wav", "")
    audio_path = f"data/dataset/audios_test/{audio_id}.wav"

    retries = 3
    for attempt in range(retries):
        try:
            result = evaluate_audio_file(audio_path)
            df_results.loc[idx, "label"] = result["score"]  # Only update the score
            break
        except Exception as e:
            print(f"[!] Attempt {attempt+1} failed for {filename}: {e}")
            if attempt < retries - 1:
                sleep_time = 2 ** attempt + random.uniform(0, 1.5)
                print(f"    ↪ Retrying in {sleep_time:.2f}s...")
                time.sleep(sleep_time)
            else:
                print(f"[X] Final failure on {filename}")


Retrying failed audios:   0%|          | 0/1 [00:00<?, ?it/s]

[+] Transcribing data/dataset/audios_test/audio_75.wav ...
[✓] Transcribed:  . 10. 11. 12. 13. 14. 15. 16. 17. 18. Lisa la Ka Pari Nesmo R Spielt Amai olasaly hurt passara Spit Terima kasih telah menonton!
[+] Sending to LLM for evaluation...
[✓] LLM Output:
Score: 1.0
Suggestions: The sentence appears to be a list of numbers and words without any grammatical structure or meaning. Consider rephrasing the sentence to convey a clear message or thought.
Mistakes: Lack of grammatical structure, no subject-verb agreement, no punctuation, no coherent meaning.


Retrying failed audios: 100%|██████████| 1/1 [00:12<00:00, 12.39s/it]


In [36]:
df_results.to_csv("final_submission.csv", index=False)
print("[✓] Saved final submission to submission.csv")


[✓] Saved final submission to submission.csv


In [35]:
# Check for NaNs or empty values in required columns
print("Missing score values:", df_results['label'].isna().sum())
print("Missing filename values:", df_results['filename'].isna().sum())



Missing score values: 0
Missing filename values: 0


In [37]:
from google.colab import files
files.download("final_submission.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### 📌 Observations

- LLaMA3 performs better with clean few-shot prompts
- Using bucketed examples across score ranges increases LLM calibration
- Whisper errors still impact the final score

### 🛠 Future Work

- Add hallucination filtering (regex rules)
- Use multiple-shot + CoT prompting
- RAG or retrieval-augmented examples based on audio metadata
